In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("hive + Jupiter").enableHiveSupport().getOrCreate()

In [0]:
spark.sql("show databases").show()

In [0]:
spark.sql("use default").show()

In [0]:
spark.sql("show tables").show()

In [0]:
spark.sql("create database practica3")

In [0]:
spark.sql("show databases").show()

In [0]:
spark.sql("use practica3")

In [0]:
spark.sql("CREATE TABLE caso_real \
(client_id INT, servicio_id INT, nombre_servicio STRING, meses_activo INT, domainstatus STRING, paymentmethod STRING, billingcycle STRING, total DOUBLE, state STRING) \
ROW FORMAT DELIMITED \
FIELDS TERMINATED BY ',' \
STORED AS TEXTFILE")

          

In [0]:
spark.sql("show tables").show()

In [0]:
spark.sql("select * from caso_real").show()

In [0]:

spark.sql("load data local inpath 'dbfs:/FileStore/practica2.csv' overwrite into table caso_real")

In [0]:
data=spark.sql("select * from caso_real")
spark.sql("select * from caso_real").show(10)

**client_id**      identificador del cliente

**servicio_id**      identificador del servicio

**nombre_servicio**    nombre del servicio

**meses_activo**     catidad de meses con el servicio

**domainstatus**     estado del servicio

**paymentmethod**  metodo de pago

**billingcycle**  tiempos de pago

**total**    costo del servicio

**state**     estado/cuidad del cliente

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# Selecciona las variables cuantitativas que deseas analizar
variables_cuantitativas = ["meses_activo", "total"]

# Configura el estilo de Seaborn
sns.set(style="whitegrid")

# Crea subplots para los histogramas
fig, axes = plt.subplots(nrows=1, ncols=len(variables_cuantitativas), figsize=(15, 5))

# Itera sobre las variables cuantitativas y crea histogramas
for i, var in enumerate(variables_cuantitativas):
    sns.distplot(df.select(var).toPandas(), bins=20, kde=True, ax=axes[i])
    axes[i].set_xlabel(var)
    axes[i].set_ylabel("Frecuencia")
    axes[i].set_title(f"Histograma de {var}")

plt.tight_layout()
plt.show()

In [0]:
# Selecciona las variables cuantitativas que deseas analizar
variables_cuantitativas = ["meses_activo", "total"]

# Convierte el DataFrame de Spark en un DataFrame de pandas
df_pandas = df.select(variables_cuantitativas).toPandas()

# Configura el estilo de Seaborn
sns.set(style="whitegrid")

# Crea subplots para los boxplots
fig, axes = plt.subplots(nrows=1, ncols=len(variables_cuantitativas), figsize=(15, 5))

# Itera sobre las variables cuantitativas y crea boxplots
for i, var in enumerate(variables_cuantitativas):
    sns.boxplot(x=var, data=df_pandas, ax=axes[i])
    axes[i].set_ylabel(var)
    axes[i].set_title(f"Boxplot de {var}")

plt.tight_layout()
plt.show()


In [0]:
# Supongamos que tienes una lista de variables categóricas que deseas graficar
variables_categoricas = ["nombre_servicio", "domainstatus", "paymentmethod", "billingcycle", "state"]

# Configura el estilo de Seaborn
sns.set(style="whitegrid")

# Itera sobre las variables categóricas y crea un gráfico de barras para cada una en celdas separadas
for var in variables_categoricas:
    plt.figure(figsize=(10, 6))  # Configura el tamaño del gráfico
    sns.countplot(data=df.toPandas(), x=var, palette="Set3")
    plt.xlabel(var)
    plt.ylabel("Frecuencia")
    plt.title(f"Gráfico de Barras para {var}")
    plt.xticks(rotation=90)  # Rota las etiquetas del eje x si es necesario
    plt.show()  # Muestra el gráfico en una celda separada

In [0]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer

In [0]:
df.columns

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Define el esquema de tu DataFrame con tipos de datos específicos
schema = StructType([
    StructField("client_id", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("payments", DoubleType(), True),
    StructField("type_DISPONENT", DoubleType(), True),
    StructField("type_OWNER", DoubleType(), True),
    StructField("frequency_POPLATEK MESICNE", DoubleType(), True),
    StructField("frequency_POPLATEK PO OBRATU", DoubleType(), True),
    StructField("frequency_POPLATEK TYDNE", DoubleType(), True)
])

# Carga los datos utilizando el esquema especificado
df_minable = spark.read.csv("dbfs:/FileStore/tablaminable.csv", header=True, schema=schema)

# Aplicar StringIndexer para convertir la columna "status" en índices numéricos
indexer = StringIndexer(inputCol="status", outputCol="label")
indexed_df = indexer.fit(df_minable).transform(df_minable)

# Lista de nombres de columnas de características
feature_cols = ['client_id',
                'amount',
                'payments',
                'type_DISPONENT',
                'type_OWNER',
                'frequency_POPLATEK MESICNE',
                'frequency_POPLATEK PO OBRATU',
                'frequency_POPLATEK TYDNE']

# Crear un VectorAssembler que ensambla las características en una columna llamada "feature_vector"
assembler = VectorAssembler(inputCols=feature_cols, outputCol="feature_vector")

# Aplicar el VectorAssembler al DataFrame
df_assembled = assembler.transform(indexed_df)

train_data, test_data = df_assembled.randomSplit([0.8, 0.2], seed=42)

# Crear un RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", featuresCol="feature_vector", numTrees=100, seed=42)

# Crear un Pipeline para encadenar la clasificación
pipeline = Pipeline(stages=[rf])

# Entrenar el modelo en los datos de entrenamiento
model = pipeline.fit(train_data)

# Realizar predicciones en los datos de prueba
predictions = model.transform(test_data)

# Métricas de clasificación multiclase
multiclass_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

# F1 Score
f1 = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName: "f1"})

# Recall
recall = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName: "weightedRecall"})

# Imprimir métricas
print("F1 Score:", f1)
print("Recall:", recall)
